In [7]:
#Importación de librerías y configuración inicial
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
import numpy as np
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer, SimpleImputer

import seaborn as sns
import os
import shutil
import json
import joblib

!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

In [8]:
#Descarga de datos desde Kaggle
os.environ['KAGGLE_CONFIG_DIR']='/content/'

data = {"username":"kellyarangohenao","key":"49fe4a9967395e59af1b5df1a5194b48"}
with open('kaggle.json','w') as file:
  json.dump(data, file, indent=4)


!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 351MB/s]


In [9]:
#Descompresión y verificación de archivos
!unzip udea*.zip > /dev/null
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


In [10]:
#Carga del conjunto de entrenamiento
file = "train.csv"
df = pd.read_csv(file)
df.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [11]:
#Imputación de valores faltantes
columnas_numericas = df.select_dtypes(include=["float64", "int64"]).columns
columnas_categoricas = df.select_dtypes(include=["object"]).columns


imputar_numericas = KNNImputer(n_neighbors=5)
df[columnas_numericas] = imputar_numericas.fit_transform(df[columnas_numericas])


imputar_categoricas = SimpleImputer(strategy="most_frequent")
df[columnas_categoricas] = imputar_categoricas.fit_transform(df[columnas_categoricas])


print("Valores faltantes después de la imputación:")
print(df.isnull().sum())

Valores faltantes después de la imputación:
ID                                0
PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_TIENELAVADORA                0
FAMI_TIENEAUTOMOVIL               0
ESTU_PRIVADO_LIBERTAD             0
ESTU_PAGOMATRICULAPROPIO          0
FAMI_TIENECOMPUTADOR              0
FAMI_TIENEINTERNET.1              0
FAMI_EDUCACIONMADRE               0
RENDIMIENTO_GLOBAL                0
coef_1                            0
coef_2                            0
coef_3                            0
coef_4                            0
dtype: int64


In [12]:
# Codificación de variables categóricas
label_encoder = LabelEncoder()
df["RENDIMIENTO_GLOBAL"] = label_encoder.fit_transform(df["RENDIMIENTO_GLOBAL"])
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = label_encoder.fit_transform(df[col])

In [13]:
# Markdown Escalado de características numéricas
scaler = StandardScaler()
df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])

print("Variables numéricas escaladas:")
print(df[columnas_numericas].head())

Variables numéricas escaladas:
         ID   PERIODO    coef_1    coef_2    coef_3    coef_4
0  1.434424  1.294094  0.437002 -0.556223  0.813978  0.060296
1  0.527513  1.294094  0.346934 -0.481341  0.508180  0.016142
2 -0.652132  0.439801  0.232301 -0.492038  0.729034  0.016142
3 -0.084924 -0.319570  1.771650 -0.941332 -0.171371 -1.072993
4  1.731274  1.294094  0.387874 -0.299484  0.389259  0.457683


In [14]:
# Tratamiento de valores atípicos (outliers)
umbral = 3


for col in columnas_numericas:
    if df[col].std() == 0:
        continue
    z_scores = zscore(df[col])
    mediana = df[col].median()
    df[col] = np.where(np.abs(z_scores) > umbral, mediana, df[col])



Valores atípicos suavizados reemplazando por la mediana usando Z-score.


In [15]:
#Separación de variables predictoras y objetivo

X = df.drop(columns=["RENDIMIENTO_GLOBAL", "ID"])
y = df["RENDIMIENTO_GLOBAL"]

In [16]:
#División del Conjunto de Datos en Entrenamiento y Validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
# Entrenamiento y Evaluación del Modelo con HistGradientBoostingClassifier
hgbboost  = HistGradientBoostingClassifier(
    max_iter=100, learning_rate=0.1, max_depth=5, random_state=42
)
hgbboost .fit(X_train, y_train)
y_pred_hgbboost  = modelo_hgbboost .predict(X_val)
accuracy = accuracy_score(y_val, y_pred_hgbboost )

print(f'Precisión: {accuracy}')

Precisión con Hist Gradient Boosting: 0.4276871239470517


In [25]:
#Carga del Conjunto de Prueba
file = "test.csv"
df_test = pd.read_csv(file)

In [26]:
#Imputación y Escalado de Datos Numéricos en el Conjunto de Prueba
df_test[columnas_numericas] = imputar_numericas.transform(df_test[columnas_numericas])
df_test[columnas_numericas] = scaler.transform(df_test[columnas_numericas])

In [27]:
#Variables Categóricas y Predicción Final sobre el Conjunto de Prueba
for col in df_test.select_dtypes(include=["object"]).columns:
    df_test[col] = label_encoder.fit_transform(df_test[col])

X_test = df_test.drop(columns=["ID"])
predicciones = modelo_hgbboost .predict(X_test)

In [28]:
#DataFrame de Envío con Predicciones Finales
df_submission = pd.DataFrame({"ID": df_test["ID"], "RENDIMIENTO_GLOBAL": predicciones})

In [29]:
# Guardar el archivo en formato CSV
df_submission.to_csv("submission.csv", index=False)
print("Archivo 'submission.csv' creado con éxito.")

Archivo 'submission.csv' creado con éxito.


In [30]:
#Envío del Archivo de Predicciones a la Competencia de Kaggle
!kaggle competitions submit -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia -f submission.csv -m "Kelly Julieth Arango Henao solucion"

100% 6.10M/6.10M [00:00<00:00, 13.4MB/s]
Successfully submitted to UDEA/ai4eng 20251 - Pruebas Saber Pro Colombia